
Tout d'abord, installer les librairies nécessaires: 


In [81]:
!pip install --quiet transformers datasets evaluate seqeval

In [82]:
import datasets

Charger les libraries dans l'environnement

In [83]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification

import zipfile

import evaluate
import numpy as np

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import pipeline


In [84]:
import numpy

Uploader dans votre environment de travail l'archive zip fournie (corpusCasM2-main.zip) à l'aide de l'explorateur de fichier google colab.

Puis décompresser le fichier avec la commande suivante: 

In [85]:
#with zipfile.ZipFile("corpusCasM2-main.zip", "r") as archive:
        #archive.extractall("data")

Charger le dataset corpusCasM2

In [86]:
#corpusCas = load_dataset("data/corpusCasM2-main/corpusCasM2")

Afficher un exemple du trainset:

id
: correspond à l'id du document et le numero de la phrase dans le document 

tokens
: contient le texte de la phrase pre-tokenisé

ner_tags
: contient les labels au format [BIO](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)). Ils sont affichés ici par leur ID 

In [87]:
corpusCas = datasets.load_dataset('D://reseau de neurone//Projet NLP//code source','test.json')

Using custom data configuration code source-473889fb0da93825
Found cached dataset json (C:/Users/mefta/.cache/huggingface/datasets/json/code source-473889fb0da93825/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)



100%|██████████| 3/3 [00:00<00:00, 59.86it/s]


In [88]:
corpusCas["train"][0]['biluo']

['O',
 'U-date',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-date',
 'L-date',
 'O',
 'B-problem',
 'L-problem',
 'O',
 'O',
 'O',
 'O',
 'B-problem',
 'L-problem',
 'O',
 'O',
 'O',
 'O',
 'B-problem',
 'I-problem',
 'I-problem',
 'I-problem',
 'I-problem',
 'L-problem',
 'O',
 'O',
 'B-problem',
 'L-problem',
 'O']

In [89]:
#my_list = list(set(corpusCas["train"][1]['biluo']))
my_list1= corpusCas["train"][:len(corpusCas["train"])]['biluo']
len(my_list1)
a1=numpy.concatenate( my_list1, axis=0 )
a1
distinct_train=set(a1)
print(distinct_train)
len(distinct_train)

compte=0
liste=corpusCas["train"][:len(corpusCas["train"])]["biluo"]
tot_mot=numpy.concatenate(liste, axis=0)
for i in range(0, len(tot_mot)):
    if tot_mot[i]=='-':
        compte=compte+1
print(compte)



{'U-date', 'I-treatment', 'B-date', 'U-treatment', 'B-duration', 'B-frequency', 'I-duration', 'L-test', 'I-test', 'L-problem', 'U-frequency', 'L-frequency', 'L-date', 'U-problem', 'I-problem', 'O', 'U-test', '-', 'I-frequency', 'B-treatment', 'U-duration', 'L-treatment', 'L-duration', 'I-date', 'B-problem', 'B-test'}
966


In [90]:
my_list=[]
for i in range(0, len(corpusCas["train"])):
    my_list.append(corpusCas["train"][i]["bio"])
my_list
distinct=numpy.concatenate(my_list, axis=0)
distinct=set(distinct)
print(distinct)

{'I-test', 'I-frequency', 'B-problem', 'B-duration', 'B-treatment', 'B-date', '0', 'B-frequency', 'I-duration', 'B-test', 'I-treatment', 'I-date', 'I-problem', 'O'}


Il est possible de récupérer la liste des labels à partir des features du dataset: 

In [91]:
# label_list = corpusCas["train"].features["bio"].feature.names
# label_list
def set_features(tag, labels):
    corpusCas["train"].features[tag].feature.names=labels
    corpusCas["test"].features[tag].feature.names=labels
    corpusCas["validation"].features[tag].feature.names=labels


In [92]:
labels=["B-date","I-date","B-duration","I-duration","B-problem","I-problem","B-treatment",
"I-treatment","B-test","I-test","B-frequency","I-frequency","O"]
set_features("bio", labels)

label_list= corpusCas["train"].features["bio"].feature.names
label_list


['B-date',
 'I-date',
 'B-duration',
 'I-duration',
 'B-problem',
 'I-problem',
 'B-treatment',
 'I-treatment',
 'B-test',
 'I-test',
 'B-frequency',
 'I-frequency',
 'O']

In [93]:
def add_ner_col(X):
    ner_tags=[]
    for i in range(0, len(corpusCas[X])):
        ner=[]
        for j in range(0, len(corpusCas[X][i]["bio"])):
            token=corpusCas[X][i]["bio"][j]
            if token=="0":
                token="O"
            ner.append(labels.index(token))
        ner_tags.append(ner)

    print(len(ner_tags))
    corpusCas[X]=corpusCas[X].add_column(name="ner", column=ner_tags)




In [94]:
add_ner_col("train")
add_ner_col("validation")
add_ner_col("test")

8305
2122
2545


In [95]:
print(corpusCas["test"][2]["bio"])
print(corpusCas["test"][2]["ner"])


['O']
[12]


In [96]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))
show_random_elements(corpusCas["train"])

,doc_id,sent_id,token,biluo,bio,pos,ner
0,517,517_015,"[\n\n, L’, exploration, per, -, opératoire, a, retrouvé, un, utérus, didelphe, (, Figure, 4, ), ,, des, ovaires, hypoplasiques, et, un, rein, droit, hypoplasique, en, ectopie, pelvienne, .]","[O, O, B-test, I-test, I-test, L-test, O, O, O, B-problem, L-problem, O, O, O, O, O, O, B-problem, L-problem, O, O, B-problem, I-problem, L-problem, O, B-test, L-test, O]","[O, O, B-test, I-test, I-test, I-test, O, O, O, B-problem, I-problem, O, O, O, O, O, O, B-problem, I-problem, O, O, B-problem, I-problem, I-problem, O, B-test, I-test, O]","[SPACE, PROPN, NOUN, ADP, PROPN, NOUN, AUX, VERB, DET, NOUN, ADJ, PUNCT, PROPN, NUM, PUNCT, PUNCT, DET, NOUN, ADJ, CCONJ, DET, NOUN, ADJ, ADJ, ADP, NOUN, ADJ, PUNCT]","[12, 12, 8, 9, 9, 9, 12, 12, 12, 4, 5, 12, 12, 12, 12, 12, 12, 4, 5, 12, 12, 4, 5, 5, 12, 8, 9, 12]"
1,618,618_020,[\n\n],[O],[O],[SPACE],[12]


Télécharger le tokenizer du modèle choisi (ici "distilbert-base-uncased"): 

In [97]:
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\mefta/.cache\huggingface\hub\models--camembert-base\snapshots\3f452b6e5a89b0e6c828c9bba2642bc577086eae\config.json
Model config CamembertConfig {
  "_name_or_path": "camembert-base",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32005
}

loading file sentencepiece.

Voici un exemple d'output du tokenizer: 

In [98]:
# example = corpusCas["train"][0]
# tokenized_input = tokenizer(example["token"], is_split_into_words=True)
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
# tokens

In [99]:
example = corpusCas["train"][0]
print(example["token"])
print()
tokenized_input = tokenizer(example["token"], is_split_into_words=True)
print(tokenized_input)
print()
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['En', '1988', ',', 'F.G.', ',', 'un', 'homme', 'de', '58', 'ans', ',', 'tabagique', 'ancien', ',', 'a', 'présenté', 'une', 'hématurie', 'totale', 'en', 'rapport', 'avec', 'une', 'tumeur', 'vésicale', 'unique', ',', 'paraméatique', 'gauche', ',', 'sans', 'anomalie', 'urographique', '.']

{'input_ids': [5, 107, 10500, 21, 7, 358, 9, 546, 9, 21, 7, 23, 421, 8, 6928, 134, 21, 7, 19334, 55, 10406, 1808, 21, 7, 33, 2186, 28, 21, 22488, 16486, 2273, 22, 459, 42, 28, 16775, 8890, 10, 6533, 35, 931, 21, 7, 37, 55, 2037, 55, 1603, 953, 21, 7, 112, 15841, 21, 297, 11136, 21, 9, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

['<s>', '▁En', '▁1988', '▁', ',', '▁F', '.', 'G', '.', '▁', ',', '▁un', '▁homme', '▁de', '▁58', '▁ans', '▁', ',', '▁tab', 'a', 'gique', '▁ancien', '▁', ',', '▁a', '▁présenté', '▁une', '▁', 'hémat', 'urie', '▁totale', '▁en',

Vous pouvez constater que le texte tokenisé comprend plus de tokens que de mots initialement présents. Or les labels n'existent que pour un mot. Il faut donc réaligner les tokens et les labels en utilisant la fonction suivante: 

In [100]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["token"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [101]:
tokenized_corpus = corpusCas.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at C:\Users\mefta\.cache\huggingface\datasets\json\code source-473889fb0da93825\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-41dd8592b9351ad2.arrow
Loading cached processed dataset at C:\Users\mefta\.cache\huggingface\datasets\json\code source-473889fb0da93825\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-e008d0134dfba622.arrow






  0%|          | 0/780 [30:51<?, ?it/s]



100%|██████████| 3/3 [00:02<00:00,  1.27ba/s]


On obtient bien l'effet escompté: 


In [102]:
tokenized_corpus["test"][1]

{'doc_id': 743,
 'sent_id': '743_001',
 'token': ['\n\n',
  'L´échographie',
  'rénale',
  'et',
  'vésicoprostatique',
  'a',
  'objectivé',
  'une',
  'urétérohydronéphrose',
  'bilatérale',
  'et',
  'la',
  'présence',
  'd’',
  'un',
  'résidu',
  'post',
  'mictionnel',
  'significatif',
  '.'],
 'biluo': ['O',
  'B-test',
  'I-test',
  'I-test',
  'L-test',
  'O',
  'O',
  'O',
  'B-problem',
  'L-problem',
  'O',
  'O',
  'O',
  'O',
  'B-problem',
  'I-problem',
  'L-problem',
  'B-problem',
  'L-problem',
  'O'],
 'bio': ['O',
  'B-test',
  'I-test',
  'I-test',
  'I-test',
  'O',
  'O',
  'O',
  'B-problem',
  'I-problem',
  'O',
  'O',
  'O',
  'O',
  'B-problem',
  'I-problem',
  'I-problem',
  'B-problem',
  'I-problem',
  'O'],
 'pos': ['SPACE',
  'PROPN',
  'ADJ',
  'CCONJ',
  'ADJ',
  'AUX',
  'VERB',
  'DET',
  'NOUN',
  'ADJ',
  'CCONJ',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADJ',
  'ADJ',
  'ADJ',
  'PUNCT'],
 'ner': [12, 8, 9, 9, 9, 12, 12, 12, 4, 5, 12

On crée ensuite un [DataCollator](https://huggingface.co/docs/transformers/main_classes/data_collator) qui va servir à créer les batchs en entrée du modèle. On utilise un DataCollator spécifique à la tache 

In [103]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Ensuite on crée une petite fonction qui va être chargée de faire l'évaluation du modèle: ici on se sert de la métrique [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval).  

In [104]:

seqeval = evaluate.load("seqeval")
seq_labels = [label_list[i] for i in example[f"ner"]]



def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Créons à présent les dictionnaires qui permettent de lier les labels avec leurs IDs respectifs


In [105]:
id2label = {i:label for i,label in enumerate(label_list)}
label2id = {v:k for k,v in id2label.items()}
label2id


{'B-date': 0,
 'I-date': 1,
 'B-duration': 2,
 'I-duration': 3,
 'B-problem': 4,
 'I-problem': 5,
 'B-treatment': 6,
 'I-treatment': 7,
 'B-test': 8,
 'I-test': 9,
 'B-frequency': 10,
 'I-frequency': 11,
 'O': 12}

Nous pouvons à présent instancier le modèle que nous voulons fine-tuner: 


In [106]:

model = AutoModelForTokenClassification.from_pretrained(
    "camembert-base", num_labels=len(id2label), id2label=id2label, label2id=label2id
)

loading configuration file config.json from cache at C:\Users\mefta/.cache\huggingface\hub\models--camembert-base\snapshots\3f452b6e5a89b0e6c828c9bba2642bc577086eae\config.json
Model config CamembertConfig {
  "_name_or_path": "camembert-base",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-date",
    "1": "I-date",
    "2": "B-duration",
    "3": "I-duration",
    "4": "B-problem",
    "5": "I-problem",
    "6": "B-treatment",
    "7": "I-treatment",
    "8": "B-test",
    "9": "I-test",
    "10": "B-frequency",
    "11": "I-frequency",
    "12": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-date": 0,
    "B-duration": 2,
    "B-frequency": 10,
    "B-problem": 4,
    "B-test": 8,
    "B-treatment": 6,
    "I-date": 1,
    "

In [107]:
batch_size=8

args = TrainingArguments(
    "camembert-base-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [108]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_corpus["train"],
    eval_dataset=tokenized_corpus["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [109]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: sent_id, doc_id, biluo, pos, bio, ner, token. If sent_id, doc_id, biluo, pos, bio, ner, token are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
c:\Users\mefta\miniconda3\envs\corpuscas\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8305
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3117
  Number of trainable parameters = 110041357
 16%|█▌        | 500/3117 [32:05<3:44:

{'loss': 1.1801, 'learning_rate': 1.679178697465512e-05, 'epoch': 0.48}


Model weights saved in camembert-base-finetuned\checkpoint-500\pytorch_model.bin
tokenizer config file saved in camembert-base-finetuned\checkpoint-500\tokenizer_config.json
Special tokens file saved in camembert-base-finetuned\checkpoint-500\special_tokens_map.json
 32%|███▏      | 1000/3117 [1:52:14<1:59:42,  3.39s/it]  

                                                         
 32%|███▏      | 1000/3117 [1:52:14<1:59:42,  3.39s/it]Saving model checkpoint to camembert-base-finetuned\checkpoint-1000
Configuration saved in camembert-base-finetuned\checkpoint-1000\config.json


{'loss': 0.6958, 'learning_rate': 1.3583573949310235e-05, 'epoch': 0.96}


Model weights saved in camembert-base-finetuned\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in camembert-base-finetuned\checkpoint-1000\tokenizer_config.json
Special tokens file saved in camembert-base-finetuned\checkpoint-1000\special_tokens_map.json
 33%|███▎      | 1039/3117 [1:54:20<1:36:11,  2.78s/it]The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: sent_id, doc_id, biluo, pos, bio, ner, token. If sent_id, doc_id, biluo, pos, bio, ner, token are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2122
  Batch size = 8
c:\Users\mefta\miniconda3\envs\corpuscas\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

{'eval_loss': 0.5905081033706665, 'eval_precision': 0.5262452107279694, 'eval_recall': 0.616887491578711, 'eval_f1': 0.5679727075364417, 'eval_accuracy': 0.8520063613524789, 'eval_runtime': 253.899, 'eval_samples_per_second': 8.358, 'eval_steps_per_second': 1.048, 'epoch': 1.0}


 42%|████▏     | 1316/3117 [2:41:07<1:24:17,  2.81s/it]   

## TODO

Pour simplifier le travail, les **scopes de temporalité ne sont pas pris en compte** dans cette version du dataset. 

1. Identifier un modèle adapté à la tâche. Ici nous voulons faire de la reconnaissance d'entités nommées (classification de tokens) dans des textes en français. Ce choix devra être argumenté. Dans l'exemple, le modèle choisi n'est pas particulièrement adapté au français. 

2. Fine-tuner ce modèle sur la tâche. C'est à dire réentrainer le modèle avec les données du corpusCasM2. Vous devrez tester différentes combinaisons d'hyperparamètres afin de trouver les meilleurs. Pour cela, vous mettrez en place une stratégie d'optimisation des hyperparamètres que vous justifierez.

3. Evaluer de manière adaptée et argumentée les résultats obtenus.
